# Calculate recall score, precision score, f1 score and roc_auc_score

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score
from sklearn.feature_selection import RFECV
from rdkit import Chem

import os
import numpy as np
import pandas as pd

from agent.rule_metric import mean_support, mean_confidence, mean_lift, mean_leverage

def calculate_metrics_with_loo(feature_matrix, labels, model_cla):
    """
    Calculate evaluation metrics (recall, precision, F1-score, and ROC-AUC score) using Leave-One-Out (LOO) cross-validation.

    Args:
        feature_matrix (numpy.ndarray): The input feature matrix.
        labels (numpy.ndarray): The corresponding labels.
        model_cla: A random forest classifier.

    Returns:
        dict: A dictionary containing recall, precision, F1-score, and ROC-AUC score.
    """
    # Initialize Leave-One-Out cross-validator
    loo = LeaveOneOut()

    # Lists to store predictions and true labels for LOO
    y_true = []
    y_pred = []
    y_pred_proba = []

    # Perform Leave-One-Out cross-validation
    for train_index, test_index in loo.split(feature_matrix):
        X_train, X_test = feature_matrix[train_index], feature_matrix[test_index]
        y_train, y_test = labels[train_index], labels[test_index]

        # Train the model
        model_cla.fit(X_train, y_train)

        # Predict on the test set
        y_pred.append(model_cla.predict(X_test)[0])
        y_pred_proba.append(model_cla.predict_proba(X_test)[0][1])  # Probability of the positive class
        y_true.append(y_test[0])

    # Calculate metrics
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred_proba)

    # Store metrics in a dictionary
    metrics = {
        "recall": recall,
        "precision": precision,
        "f1_score": f1,
        "roc_auc_score": roc_auc,
    }

    return metrics

## Traditional descriptor

In [13]:
# Load the target column
target = 'yield'
df = pd.read_csv("agent/data/data.csv")[[target]]

# Calculate the median of the target column
median_value = df[target].median()

# Transform the target column into 0/1 based on the median
df[target] = (df[target] > median_value).astype(int)



# Load the dataset
dataset = pd.read_csv("agent/data/data.csv")
feature_file = "agent/data/del_and_exp_features.csv"
selected_features = ['NumHDonors','Solvent_area','Conju_LogP_Std']

label = df[target].values
feature = pd.read_csv(feature_file)[selected_features]
feature = feature.values

model_cla = RandomForestClassifier(n_estimators=500, max_depth=2, max_features=None, random_state=42, n_jobs=64)
metrics = calculate_metrics_with_loo(feature,label,model_cla)
print('Traditional Descriptor')
print(metrics)


Traditional Descriptor
{'recall': 0.4444444444444444, 'precision': 0.6153846153846154, 'f1_score': 0.5161290322580645, 'roc_auc_score': 0.5246913580246914}


## Fe loading

In [16]:
# Load the target column
target = 'yield'
df = pd.read_csv("agent/data/data.csv")[[target]]

# Calculate the median of the target column
median_value = df[target].median()

# Transform the target column into 0/1 based on the median
df[target] = (df[target] > median_value).astype(int)


# Load the dataset
dataset = pd.read_csv("agent/data/data.csv")
# Convert the labels to numpy array
label = df[target].values
feature = dataset[['Fe_loading']]
feature = feature.values

model_cla = RandomForestClassifier(n_estimators=500, max_depth=1, max_features=None, random_state=42, n_jobs=64)
metrics = calculate_metrics_with_loo(feature,label,model_cla)
print('Fe loading')
print(metrics)

Fe loading
{'recall': 0.6666666666666666, 'precision': 0.75, 'f1_score': 0.7058823529411765, 'roc_auc_score': 0.7839506172839505}


## Exp. descriptor

In [15]:
# Load the target column
target = 'yield'
df = pd.read_csv("agent/data/data.csv")[[target]]

# Calculate the median of the target column
median_value = df[target].median()

# Transform the target column into 0/1 based on the median
df[target] = (df[target] > median_value).astype(int)


# Load the dataset
dataset = pd.read_csv("agent/data/data.csv")
# Convert the labels to numpy array
label = df[target].values
feature = dataset[['Fe_loading','modifier/SBU']]
feature = feature.values


model_cla = RandomForestClassifier(n_estimators=500, max_depth=2, max_features=None, random_state=42, n_jobs=64)
metrics = calculate_metrics_with_loo(feature,label,model_cla)
print('Exp. Descriptor')
print(metrics)


Exp. Descriptor
{'recall': 0.6666666666666666, 'precision': 0.75, 'f1_score': 0.7058823529411765, 'roc_auc_score': 0.7685185185185186}


## qwen-max (Direct) + Exp. Descriptor

In [20]:
# rules generate by qwen-max-2025-01-25 looking through the paper
def rule2matrix(smiles_list):
    # Define SMARTS patterns for functional groups and structural features
    aromatic_ring = 'c1ccccc1'
    hydroxyl_group = '[OX2H]'
    alkoxy_group = '[OX2][#6]'
    amino_group = '[NX3;H2,H1][#6]'
    nitro_group = '[NX3](=O)=O'
    halogen_group = '[F,Cl,Br,I]'
    trifluoromethyl_group = '[CX4][F][F][F]'
    carboxylic_acid = '[CX3](=O)[OX2H1]'
    carbonyl_group = '[CX3]=[OX1]'
    aldehyde_group = '[CX3H1](=O)'
    bulky_group = '[CX4;!$([CX4][C])][CX4;!$([CX4][C])]'
    long_aliphatic_chain = '[CH2][CH2][CH2][CH2][CH2]'

    # Define the rules with their associated patterns and predictions
    rules = [
        {
            'number': 1,
            'description': 'High Yield: Aromatic ring present.',
            'patterns': [[aromatic_ring]],
            'prediction': 1
        },
        {
            'number': 2,
            'description': 'High Yield: Hydroxyl group present.',
            'patterns': [[hydroxyl_group]],
            'prediction': 1
        },
        {
            'number': 3,
            'description': 'High Yield: Alkoxy group present.',
            'patterns': [[alkoxy_group]],
            'prediction': 1
        },
        {
            'number': 4,
            'description': 'High Yield: Amino group present.',
            'patterns': [[amino_group]],
            'prediction': 1
        },
        {
            'number': 5,
            'description': 'Low Yield: Nitro group present.',
            'patterns': [[nitro_group]],
            'prediction': -1
        },
        {
            'number': 6,
            'description': 'Low Yield: Halogen group present.',
            'patterns': [[halogen_group]],
            'prediction': -1
        },
        {
            'number': 7,
            'description': 'Low Yield: Trifluoromethyl group present.',
            'patterns': [[trifluoromethyl_group]],
            'prediction': -1
        },
        {
            'number': 8,
            'description': 'Low Yield: Bulky functional groups present.',
            'patterns': [[bulky_group]],
            'prediction': -1
        },
        {
            'number': 9,
            'description': 'Low Yield: Long aliphatic chains present.',
            'patterns': [[long_aliphatic_chain]],
            'prediction': -1
        },
        {
            'number': 10,
            'description': 'High Yield: Carboxylic acid group present.',
            'patterns': [[carboxylic_acid]],
            'prediction': 1
        },
        {
            'number': 11,
            'description': 'Low Yield: Carbonyl or aldehyde group present.',
            'patterns': [[carbonyl_group, aldehyde_group]],
            'prediction': -1
        }
    ]

    # Compile SMARTS patterns
    for rule in rules:
        compiled_patterns = []
        for group in rule.get('patterns', []):
            compiled_group = [Chem.MolFromSmarts(p) for p in group]
            compiled_patterns.append(compiled_group)
        rule['compiled_patterns'] = compiled_patterns

    # Initialize results list
    results = []

    # Process each SMILES string
    for smi in smiles_list:
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            # If the molecule cannot be parsed, append a row of zeros
            results.append([0] * len(rules))
            continue
        row = []
        for rule in rules:
            try:
                match = False
                # Check required patterns
                for compiled_group in rule['compiled_patterns']:
                    group_match = False
                    for pat in compiled_group:
                        if mol.HasSubstructMatch(pat):
                            group_match = True
                            break
                    if not group_match:
                        match = False
                        break
                    match = True
                if match:
                    row.append(rule['prediction'])
                else:
                    row.append(0)
            except Exception as e:
                # In case of any error, append 0
                row.append(0)
        results.append(row)

    # Create DataFrame with results
    df = pd.DataFrame(results, columns=[f'Rule {rule['number']}' for rule in rules])
    return df

# Load the target column
target = 'yield'
df = pd.read_csv("agent/data/data.csv")[[target]]

# Calculate the median of the target column
median_value = df[target].median()

# Transform the target column into 0/1 based on the median
df[target] = (df[target] > median_value).astype(int)

# Load the dataset
dataset = pd.read_csv("agent/data/data.csv")
feature_df = rule2matrix(list(dataset['SMILES']))

# Initialize the model
label = df[target].values
exp_feature = dataset[['Fe_loading','modifier/SBU']]
feature = pd.concat([feature_df,exp_feature],axis=1)
feature = feature.values

loo = LeaveOneOut()
model_rf = RandomForestClassifier(n_estimators=100, max_depth=4,random_state=42, n_jobs=64)
rfecv = RFECV(estimator=model_rf, step=1, cv=loo, scoring='accuracy', n_jobs=64)  # 5-fold CV for feature selection
rfecv.fit(feature, label)
# Get the selected features
selected_features = rfecv.support_  # Boolean mask of selected features
print(f"Number of selected features: {sum(selected_features)}")
print(f"Selected features: {np.where(selected_features)[0]}")  # Indices of selected features
# Reduce the feature set to the selected ones
feature = feature[:, selected_features]

if feature.shape[1] <= 4:
    depth = feature.shape[1]
else:
    depth = 4

model_cla = RandomForestClassifier(n_estimators=500, max_depth=depth, max_features=None, random_state=42, n_jobs=64)
metrics = calculate_metrics_with_loo(feature,label,model_cla)
print('qwen-max (Direct) + Exp. Descriptor')
print(metrics)

Number of selected features: 2
Selected features: [11 12]
qwen-max (Direct) + Exp. Descriptor
{'recall': 0.6666666666666666, 'precision': 0.75, 'f1_score': 0.7058823529411765, 'roc_auc_score': 0.7685185185185186}


## imi-k1.5 (Direct) + Exp. Descriptor

In [21]:
# rules generate by Kimi-k1.5 looking through the paper
def rule2matrix(smiles_list):
    # Define SMARTS patterns for functional groups
    # Aromatic ring
    aromatic_ring = '[a]'

    # Electron-Withdrawing Groups (EWGs)
    nitro_group = '[NX3](=O)=O'
    halogen = '[F,Cl,Br,I]'
    trifluoromethyl = '[CX4]([F])([F])[F]'
    
    # EWG attached to aromatic ring
    ewg_on_aromatic = ['[a][NX3](=O)=O', '[a][F,Cl,Br,I]', '[a][CX4]([F])([F])[F]']
    
    # Carboxylic acid group
    carboxylic_acid = '[CX3](=O)[OX2H1]'
    
    # Electron-Donating Groups (EDGs)
    hydroxyl_group = '[OX2H]'
    amino_group = '[NX3H2]'
    methoxy_group = '[OX2][CH3]'
    edg_on_aromatic = ['[a][OX2H]', '[a][NX3H2]', '[a][OX2][CH3]']

    # Aldehyde group
    aldehyde = '[CX3H][OX1]'

    # Define the rules
    rules = [
        {
            'number': 1,
            'description': 'High yield (+1): Modifiers containing aromatic rings with electron-withdrawing groups attached and connected to a carboxylic acid group.',
            'patterns': [
                ewg_on_aromatic,  # EWG on aromatic ring
                [carboxylic_acid]  # Carboxylic acid group
            ],
            'prediction': 1
        },
        {
            'number': 2,
            'description': 'High yield (+1): Modifiers containing more than one carboxylic acid group.',
            'patterns': [
                [carboxylic_acid]
            ],
            'count_threshold': { carboxylic_acid: 2 },
            'prediction': 1
        },
        {
            'number': 3,
            'description': 'Low yield (-1): Modifiers containing electron-donating groups connected to aromatic rings along with a carboxylic acid group.',
            'patterns': [
                edg_on_aromatic,  # EDG on aromatic ring
                [carboxylic_acid]
            ],
            'prediction': -1
        },
        {
            'number': 4,
            'description': 'Low yield (-1): Modifiers that are bulky or highly branched structures (e.g., having more than one ring).',
            'patterns': [],
            'check_ring_count': 2,
            'prediction': -1
        },
        {
            'number': 5,
            'description': 'Low yield (-1): Modifiers containing aldehyde group along with carboxylic acid group.',
            'patterns': [
                [aldehyde],
                [carboxylic_acid]
            ],
            'prediction': -1
        }
    ]
    # Compile SMARTS patterns
    for rule in rules:
        compiled_patterns = []
        for group in rule.get('patterns', []):
            compiled_group = [Chem.MolFromSmarts(p) for p in group]
            compiled_patterns.append(compiled_group)
        rule['compiled_patterns'] = compiled_patterns

    # Initialize results list
    results = []
    
    for smi in smiles_list:
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            results.append([0]*len(rules))
            continue
        row = []
        try:
            ring_count = mol.GetRingInfo().NumRings()
        except:
            ring_count = 0
        for rule in rules:
            match = True
            # Check for ring count if specified in rule
            if 'check_ring_count' in rule:
                if ring_count >= rule['check_ring_count']:
                    row.append(rule['prediction'])
                else:
                    row.append(0)
                continue
            # Check required patterns
            for compiled_group in rule['compiled_patterns']:
                group_match = False
                for pat in compiled_group:
                    if pat is None:
                        continue
                    matches = mol.GetSubstructMatches(pat)
                    if matches:
                        # If a count threshold is specified for this pattern
                        if 'count_threshold' in rule and pat in [Chem.MolFromSmarts(k) for k in rule['count_threshold']]:
                            smarts = Chem.MolToSmarts(pat)
                            threshold = rule['count_threshold'][smarts]
                            if len(matches) >= threshold:
                                group_match = True
                                break
                        else:
                            group_match = True
                            break
                if not group_match:
                    match = False
                    break
            if match:
                row.append(rule['prediction'])
            else:
                row.append(0)
        results.append(row)
    # Create DataFrame with results
    df = pd.DataFrame(results, columns=[f"Rule {rule['number']}" for rule in rules])
    return df

# Load the target column
target = 'yield'
df = pd.read_csv("agent/data/data.csv")[[target]]

# Calculate the median of the target column
median_value = df[target].median()

# Transform the target column into 0/1 based on the median
df[target] = (df[target] > median_value).astype(int)

# Load the dataset
dataset = pd.read_csv("agent/data/data.csv")
feature_df = rule2matrix(list(dataset['SMILES']))

# Initialize the model
label = df[target].values
exp_feature = dataset[['Fe_loading','modifier/SBU']]
feature = pd.concat([feature_df,exp_feature],axis=1)
feature = feature.values

loo = LeaveOneOut()
model_rf = RandomForestClassifier(n_estimators=100, max_depth=4,random_state=42, n_jobs=64)
rfecv = RFECV(estimator=model_rf, step=1, cv=loo, scoring='accuracy', n_jobs=64)  # 5-fold CV for feature selection
rfecv.fit(feature, label)
# Get the selected features
selected_features = rfecv.support_  # Boolean mask of selected features
print(f"Number of selected features: {sum(selected_features)}")
print(f"Selected features: {np.where(selected_features)[0]}")  # Indices of selected features
# Reduce the feature set to the selected ones
feature = feature[:, selected_features]

if feature.shape[1] <= 4:
    depth = feature.shape[1]
else:
    depth = 4

model_cla = RandomForestClassifier(n_estimators=500, max_depth=depth, max_features=None, random_state=42, n_jobs=64)
metrics = calculate_metrics_with_loo(feature,label,model_cla)
print('kimi-k1.5 (Direct) + Exp. Descriptor')
print(metrics)

Number of selected features: 2
Selected features: [5 6]
kimi-k1.5 (Direct) + Exp. Descriptor
{'recall': 0.6666666666666666, 'precision': 0.75, 'f1_score': 0.7058823529411765, 'roc_auc_score': 0.7685185185185186}


## o3-mini (Direct) + Exp. Descriptor

In [22]:
# rules generate by o3-mini-high after looking through the paper
def rule2matrix(smiles_list):
    # Define SMARTS patterns
    # Carboxylic acid group
    carboxylic_acid_patterns = ['[CX3](=O)[OX2H1]', '[CX3](=O)[O-]']

    # Thiol group (-SH)
    thiol_pattern = '[SX2H]'

    # Aldehyde group (-CHO)
    aldehyde_pattern = '[$([CX3H][#6]),$([CX3H2])]=[OX1]'
    
    # Ether linkage in aliphatic chains
    ether_linkage_pattern = '[#6][OX2][#6]'

    # Halogens
    halogen_pattern = '[F,Cl,Br,I]'
    
    # Nitro group
    nitro_group_pattern = '[NX3](=O)[O-]'
    
    # Aromatic ring with halogen or nitro substituents
    halogen_on_aromatic_pattern = '[c][F,Cl,Br,I]'
    nitro_on_aromatic_pattern = '[c][NX3](=O)[O-]'

    # Primary amine (-NH2)
    primary_amine_pattern = '[NX3H2]'

    # Aromatic ring pattern
    aromatic_ring_pattern = '[a]1[a][a][a][a][a]1'
    
    # Define the rules with their associated patterns and predictions
    rules = [
        {
            'number': 1,
            'description': 'Low yield (-1): Modifiers containing a thiol group (-SH).',
            'patterns': [
                [thiol_pattern]
            ],
            'prediction': -1
        },
        {
            'number': 2,
            'description': 'Low yield (-1): Modifiers containing an aldehyde group (-CHO).',
            'patterns': [
                [aldehyde_pattern]
            ],
            'prediction': -1
        },
        {
            'number': 3,
            'description': 'Low yield (-1): Modifiers containing polyether-type structures (chains of ethoxy units).',
            'count_function': lambda mol: len(mol.GetSubstructMatches(Chem.MolFromSmarts(ether_linkage_pattern))) >= 3,
            'prediction': -1
        },
        {
            'number': 4,
            'description': 'High yield (+1): Modifiers that are simple carboxylic acids, whether aliphatic or aromatic with modest substituents (e.g., halogen or nitro groups).',
            'patterns': [
                carboxylic_acid_patterns
            ],
            'additional_patterns': [
                [halogen_on_aromatic_pattern, nitro_on_aromatic_pattern]
            ],
            'prediction': 1
        },
        {
            'number': 5,
            'description': 'High yield (+1): Modifiers that are amino acid derivatives (contain both amine and carboxylic acid groups).',
            'patterns': [
                carboxylic_acid_patterns,
                [primary_amine_pattern]
            ],
            'exclude_patterns': [
                [aromatic_ring_pattern]
            ],
            'prediction': 1
        }
    ]
    
    # Compile SMARTS patterns
    for rule in rules:
        if 'patterns' in rule:
            compiled_patterns = []
            for group in rule['patterns']:
                compiled_group = [Chem.MolFromSmarts(p) for p in group]
                compiled_patterns.append(compiled_group)
            rule['compiled_patterns'] = compiled_patterns
        if 'additional_patterns' in rule:
            compiled_additional = []
            for group in rule['additional_patterns']:
                compiled_group = [Chem.MolFromSmarts(p) for p in group]
                compiled_additional.append(compiled_group)
            rule['compiled_additional_patterns'] = compiled_additional
        if 'exclude_patterns' in rule:
            compiled_excludes = []
            for group in rule['exclude_patterns']:
                compiled_group = [Chem.MolFromSmarts(p) for p in group]
                compiled_excludes.append(compiled_group)
            rule['compiled_exclude_patterns'] = compiled_excludes
    
    # Initialize results list
    results = []
    
    # Process each SMILES string
    for smi in smiles_list:
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            # If the molecule cannot be parsed, append a row of zeros
            results.append([0]*len(rules))
            continue
        row = []
        for rule in rules:
            try:
                match = False
                # For rules with count_function (Rule 3)
                if 'count_function' in rule:
                    if rule['count_function'](mol):
                        match = True
                else:
                    # Check required patterns
                    match = True
                    for compiled_group in rule.get('compiled_patterns', []):
                        group_match = False
                        for pat in compiled_group:
                            if mol.HasSubstructMatch(pat):
                                group_match = True
                                break
                        if not group_match:
                            match = False
                            break
                    # Check additional patterns if any (used in Rule 4)
                    if match and 'compiled_additional_patterns' in rule:
                        additional_match = False
                        for compiled_group in rule['compiled_additional_patterns']:
                            for pat in compiled_group:
                                if mol.HasSubstructMatch(pat):
                                    additional_match = True
                                    break
                            if additional_match:
                                break
                        if not additional_match:
                            match = False
                    # Check exclude patterns if any
                    if match and 'compiled_exclude_patterns' in rule:
                        for compiled_group in rule['compiled_exclude_patterns']:
                            for pat in compiled_group:
                                if mol.HasSubstructMatch(pat):
                                    match = False
                                    break
                            if not match:
                                break
                if match:
                    row.append(rule['prediction'])
                else:
                    row.append(0)
            except Exception as e:
                # In case of any error, append 0
                row.append(0)
        results.append(row)
    # Create DataFrame with results
    df = pd.DataFrame(results, columns=[f'Rule {rule["number"]}' for rule in rules])
    return df

# Load the target column
target = 'yield'
df = pd.read_csv("agent/data/data.csv")[[target]]

# Calculate the median of the target column
median_value = df[target].median()

# Transform the target column into 0/1 based on the median
df[target] = (df[target] > median_value).astype(int)

# Load the dataset
dataset = pd.read_csv("agent/data/data.csv")
feature_df = rule2matrix(list(dataset['SMILES']))

# Initialize the model
label = df[target].values
exp_feature = dataset[['Fe_loading','modifier/SBU']]
feature = pd.concat([feature_df,exp_feature],axis=1)
feature = feature.values

loo = LeaveOneOut()
model_rf = RandomForestClassifier(n_estimators=100, max_depth=4,random_state=42, n_jobs=64)
rfecv = RFECV(estimator=model_rf, step=1, cv=loo, scoring='accuracy', n_jobs=64)  # 5-fold CV for feature selection
rfecv.fit(feature, label)
# Get the selected features
selected_features = rfecv.support_  # Boolean mask of selected features
print(f"Number of selected features: {sum(selected_features)}")
print(f"Selected features: {np.where(selected_features)[0]}")  # Indices of selected features
# Reduce the feature set to the selected ones
feature = feature[:, selected_features]

if feature.shape[1] <= 4:
    depth = feature.shape[1]
else:
    depth = 4

model_cla = RandomForestClassifier(n_estimators=500, max_depth=depth, max_features=None, random_state=42, n_jobs=64)
metrics = calculate_metrics_with_loo(feature,label,model_cla)
print('o3-mini (Direct) + Exp. Descriptor')
print(metrics)

Number of selected features: 6
Selected features: [0 1 2 4 5 6]
o3-mini (Direct) + Exp. Descriptor
{'recall': 0.6666666666666666, 'precision': 0.75, 'f1_score': 0.7058823529411765, 'roc_auc_score': 0.7561728395061729}


## o1 (Direct) + Exp. Descriptor

In [23]:


def rule2matrix(smiles_list):


    # Define SMARTS patterns for functional groups

    # Carboxylic acid group (-COOH)
    carboxylic_acid = '[CX3](=O)[OX1H0-,OX2H1]'

    # Benzoic acid group
    benzoic_acid = 'c1ccccc1C(=O)[O;H1,-]'

    # Halogen attached to aromatic ring
    halogen_on_aromatic = '[c][F,Cl,Br,I]'

    # Nitro group attached to aromatic ring
    nitro_on_aromatic = '[c][N+](=O)[O-]'

    # Amino group attached to aromatic ring
    amino_on_aromatic = '[c][NH2]'

    # Aldehyde group attached to aromatic ring
    aldehyde_on_aromatic = '[c][CH=O]'

    # Thiol group attached to aromatic ring
    thiol_on_aromatic = '[c][SH]'

    # Pattern for aliphatic chain of five carbons
    aliphatic_chain_five_carbons = '[CH2][CH2][CH2][CH2][CH3]'

    # Thiol group in aliphatic chain (e.g., 3-mercaptopropionic acid)
    aliphatic_thiol = '[#6][#6][SX2H]'

    # PEG-like chain (multiple ether linkages)
    peg_chain = '[#6][OX2][#6][OX2][#6][OX2][#6]'

    # Amino acid backbone pattern
    amino_acid = '[NX3;H2][CX4][CX3](=O)[O;H1,-]'

    # Phenolate group (deprotonated phenol)
    phenolate = '[c][O-]'

    # Define the rules
    rules = [
        {
            'number': 1,
            'description': 'High Yield: Modifiers containing para-substituted benzoic acids with substituents –Br, –NO₂, –NH₂ at para position.',
            'patterns': [
                [benzoic_acid],  # Benzoic acid
                [halogen_on_aromatic, nitro_on_aromatic, amino_on_aromatic]  # Substituents
            ],
            'prediction': 1
        },
        {
            'number': 2,
            'description': 'Low Yield: Modifiers containing para-substituted benzoic acids with substituents –CHO, –SH at para position.',
            'patterns': [
                [benzoic_acid],  # Benzoic acid
                [aldehyde_on_aromatic, thiol_on_aromatic]  # Substituents
            ],
            'prediction': -1
        },
        {
            'number': 3,
            'description': 'High Yield: Simple aliphatic carboxylic acids without additional polar functional groups near the metal site.',
            'patterns': [
                [carboxylic_acid],  # Carboxylic acid group
                [aliphatic_chain_five_carbons]  # Aliphatic chain of five carbons
            ],
            'exclude_patterns': [
                ['[NX3]', '[OX2]', '[SX2]']  # Exclude N, O, S (excluding the carboxylic acid oxygen)
            ],
            'prediction': 1
        },
        {
            'number': 4,
            'description': 'High Yield: Modifiers with thiol groups (-SH) far from the carboxyl group (e.g., in aliphatic chain).',
            'patterns': [
                [carboxylic_acid],  # Carboxylic acid
                [aliphatic_thiol]  # Thiol group in aliphatic chain
            ],
            'prediction': 1
        },
        {
            'number': 5,
            'description': 'Low Yield: Modifiers with multiple polar functional groups or strongly coordinating moieties (e.g., PEG-like chains).',
            'patterns': [
                [carboxylic_acid],  # Carboxylic acid
                [peg_chain]  # PEG-like chain
            ],
            'prediction': -1
        },
        {
            'number': 6,
            'description': 'High Yield: Modifiers are amino acids or related molecules (e.g., aspartic acid).',
            'patterns': [
                [carboxylic_acid],  # Carboxylic acid
                [amino_acid]  # Amino acid backbone
            ],
            'prediction': 1
        },
        {
            'number': 7,
            'description': 'Low Yield: Modifiers that directly coordinate Fe (e.g., phenolate, –SH on aromatic ring, –CHO).',
            'patterns': [
                [carboxylic_acid],
                [phenolate, thiol_on_aromatic, aldehyde_on_aromatic]  # Strong Fe-coordinating groups
            ],
            'prediction': -1
        }
    ]

    # Compile SMARTS patterns
    for rule in rules:
        compiled_patterns = []
        for group in rule.get('patterns', []):
            compiled_group = [Chem.MolFromSmarts(p) for p in group]
            compiled_patterns.append(compiled_group)
        rule['compiled_patterns'] = compiled_patterns
        # Compile exclude patterns if any
        if 'exclude_patterns' in rule:
            compiled_excludes = []
            for group in rule['exclude_patterns']:
                compiled_group = [Chem.MolFromSmarts(p) for p in group]
                compiled_excludes.append(compiled_group)
            rule['compiled_exclude_patterns'] = compiled_excludes

    # Initialize results list
    results = []

    # Process each SMILES string
    for smi in smiles_list:
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            # If the molecule cannot be parsed, append a row of zeros
            results.append([0]*len(rules))
            continue
        row = []
        for rule in rules:
            try:
                match = True
                # Check exclude patterns if any
                if 'exclude_patterns' in rule:
                    for group in rule['compiled_exclude_patterns']:
                        for pat in group:
                            if mol.HasSubstructMatch(pat):
                                match = False
                                break
                        if not match:
                            break
                    if not match:
                        row.append(0)
                        continue
                # Check required patterns
                for compiled_group in rule['compiled_patterns']:
                    group_match = False
                    for pat in compiled_group:
                        if mol.HasSubstructMatch(pat):
                            group_match = True
                            break
                    if not group_match:
                        match = False
                        break
                if match:
                    row.append(rule['prediction'])
                else:
                    row.append(0)
            except Exception:
                # In case of any error, append 0
                row.append(0)
        results.append(row)
    # Create DataFrame with results
    df = pd.DataFrame(results, columns=[f'Rule {rule['number']}' for rule in rules])
    return df

# Load the target column
target = 'yield'
df = pd.read_csv("agent/data/data.csv")[[target]]

# Calculate the median of the target column
median_value = df[target].median()

# Transform the target column into 0/1 based on the median
df[target] = (df[target] > median_value).astype(int)

# Load the dataset
dataset = pd.read_csv("agent/data/data.csv")
feature_df = rule2matrix(list(dataset['SMILES']))

# Initialize the model
label = df[target].values
exp_feature = dataset[['Fe_loading','modifier/SBU']]
feature = pd.concat([feature_df,exp_feature],axis=1)
feature = feature.values

loo = LeaveOneOut()
model_rf = RandomForestClassifier(n_estimators=100, max_depth=4,random_state=42, n_jobs=64)
rfecv = RFECV(estimator=model_rf, step=1, cv=loo, scoring='accuracy', n_jobs=64)  # 5-fold CV for feature selection
rfecv.fit(feature, label)
# Get the selected features
selected_features = rfecv.support_  # Boolean mask of selected features
print(f"Number of selected features: {sum(selected_features)}")
print(f"Selected features: {np.where(selected_features)[0]}")  # Indices of selected features
# Reduce the feature set to the selected ones
feature = feature[:, selected_features]

if feature.shape[1] <= 4:
    depth = feature.shape[1]
else:
    depth = 4

model_cla = RandomForestClassifier(n_estimators=500, max_depth=depth, max_features=None, random_state=42, n_jobs=64)
metrics = calculate_metrics_with_loo(feature,label,model_cla)
print('o1 (Direct) + Exp. Descriptor')
print(metrics)

[21:13:42] SMARTS Parse Error: syntax error while parsing: [c][CH=O]
[21:13:42] SMARTS Parse Error: Failed parsing SMARTS '[c][CH=O]' for input: '[c][CH=O]'
[21:13:42] SMARTS Parse Error: syntax error while parsing: [c][CH=O]
[21:13:42] SMARTS Parse Error: Failed parsing SMARTS '[c][CH=O]' for input: '[c][CH=O]'


Number of selected features: 6
Selected features: [0 3 5 6 7 8]
o1 (Direct) + Exp. Descriptor
{'recall': 0.7222222222222222, 'precision': 0.7647058823529411, 'f1_score': 0.7428571428571429, 'roc_auc_score': 0.7962962962962963}


## o1 (Code) + Exp. Descriptor

In [28]:
DIR = 'yield_o1'
target_name = 'yield'

all_smiles = list(pd.read_csv("agent/data/data.csv")['SMILES'])
all_best_features = pd.DataFrame({
    'Index':[x for x in range(1,37)]
})


for _ in range(1, 37):
    print(f'LOO {_}')
    folder = os.path.join(DIR,str(_))
    code_file = os.path.join(folder,"best_rule_code.txt")
    with open(code_file) as f:
        code = f.read()
    exec(code, globals())
    df = rule2matrix(all_smiles) # type: ignore
    selected_train_matrix = pd.read_csv(os.path.join(folder,'best_train_matrix.csv'))
    selected_test_matrix = pd.read_csv(os.path.join(folder,'best_test_matrix.csv'))
    # # model_cla = ExtraTreesClassifier(n_estimators=500, random_state=seed,n_jobs=64)
    selected_rules = list(selected_train_matrix.columns.values)
    df = df[selected_rules]
    # print(selected_rules)
    rule_title = ['LOO_'+str(_)+"_"+x for x in selected_train_matrix.columns.values]
    df.columns = rule_title
    all_best_features = pd.concat([all_best_features,df],axis=1)


all_best_features = all_best_features.iloc[:,1:]

# Function to find groups of columns with identical values
def find_duplicate_columns(df):
    # Create a dictionary to map unique column values to column names
    value_to_columns = {}
    for col in df.columns:
        col_values = tuple(df[col])  # Convert column values to a tuple (hashable)
        if col_values in value_to_columns:
            value_to_columns[col_values].append(col)
        else:
            value_to_columns[col_values] = [col]

    # Extract groups of column names with identical values
    duplicate_columns = {
        key: value for key, value in value_to_columns.items() if len(value) > 1
    }
    return duplicate_columns

# Apply the function to the all_best_features DataFrame
duplicate_columns = find_duplicate_columns(all_best_features)

# Print groups of duplicate columns
if duplicate_columns:
    print("Groups of columns with identical values:")
    for group in duplicate_columns.values():
        print(group)
else:
    print("No duplicate columns found.")
    

# Function to find groups of columns with identical values
def find_duplicate_columns(df):
    value_to_columns = {}
    for col in df.columns:
        col_values = tuple(df[col])  # Convert column values to a tuple (hashable)
        if col_values in value_to_columns:
            value_to_columns[col_values].append(col)
        else:
            value_to_columns[col_values] = [col]

    # Extract groups of column names with identical values
    duplicate_columns = {
        key: value for key, value in value_to_columns.items() if len(value) > 1
    }
    return duplicate_columns

# Find duplicate columns in the all_best_features DataFrame
duplicate_columns = find_duplicate_columns(all_best_features)

# Load the SHAP data
shap_data = pd.read_csv("o1_rules_SHAP_value/mean_shap_values.csv")

# Calculate absolute mean SHAP values
shap_data['Abs_Mean_SHAP_Value'] = shap_data['Mean_SHAP_Value'].abs()

# Sort by absolute mean SHAP value and get the top 20 features
top_20_shap = shap_data.nlargest(20, 'Abs_Mean_SHAP_Value')

# Extract the feature names from the top 20 SHAP values
top_20_features = top_20_shap['Rule'].tolist()

# Check if any top 20 features are in the same group of duplicate columns
overlapping_features = []

for group in duplicate_columns.values():
    overlap = list(set(group) & set(top_20_features))
    if len(overlap) > 1:  # More than one feature from the same group
        overlapping_features.append(overlap)

# Print the results
if overlapping_features:
    print("Groups of top 20 features in the same duplicate group:")
    for group in overlapping_features:
        print(group)
else:
    print("No top 20 features are in the same duplicate group.")
    
    

# Load the target column
target = 'yield'
df = pd.read_csv("agent/data/data.csv")[[target]]

# Calculate the median of the target column
median_value = df[target].median()

# Transform the target column into 0/1 based on the median
df[target] = (df[target] > median_value).astype(int)

# Verify the transformation
print(f"Median value of '{target}': {median_value}")
print(df.head())



# Load the dataset
dataset = pd.read_csv("agent/data/data.csv")


# Convert the labels to numpy array
label = df[target].values

# Read the feature names back into a list
with open('o1_rules_SHAP_value/top_20_shap_features.txt', 'r') as f:
    top_20_features_from_file = [line.strip() for line in f]
# Remove specific items from the list
# top_20_features_from_file = [feature for feature in top_20_features_from_file if feature not in ['LOO_11_Rule 7', 'LOO_27_Rule 8']]
# Select the columns corresponding to the top 20 features
top_20_features_data = all_best_features[top_20_features_from_file]

feature = dataset[['Fe_loading','modifier/SBU']]
# feature = pd.concat([feature,top_20_features_data.iloc[:, :4]],axis=1)
feature = pd.concat([feature,top_20_features_data[['LOO_30_Rule 2']]],axis=1)
feature = feature.values


model_cla = RandomForestClassifier(n_estimators=500, max_depth=3, max_features=None, random_state=42, n_jobs=64)
metrics = calculate_metrics_with_loo(feature,label,model_cla)
print('o1 (Code) + Exp. Descriptor')
print(metrics)

LOO 1
LOO 2
LOO 3
LOO 4
LOO 5
LOO 6
LOO 7


[21:16:25] SMARTS Parse Error: syntax error while parsing: [NX3][CX4][CX4]{1,3}[CX3](=O)[O;H1,H0-]
[21:16:25] SMARTS Parse Error: Failed parsing SMARTS '[NX3][CX4][CX4]{1,3}[CX3](=O)[O;H1,H0-]' for input: '[NX3][CX4][CX4]{1,3}[CX3](=O)[O;H1,H0-]'
[21:16:25] SMARTS Parse Error: syntax error while parsing: [SX2H][CX4][CX2,CX4]{0,4}[CX3](=O)[O;H1,H0-]
[21:16:25] SMARTS Parse Error: Failed parsing SMARTS '[SX2H][CX4][CX2,CX4]{0,4}[CX3](=O)[O;H1,H0-]' for input: '[SX2H][CX4][CX2,CX4]{0,4}[CX3](=O)[O;H1,H0-]'
[21:16:25] SMARTS Parse Error: syntax error while parsing: [OX2][CX2,CX3]([OX2][CX2,CX3]){1,}[O,N;H1]
[21:16:25] SMARTS Parse Error: Failed parsing SMARTS '[OX2][CX2,CX3]([OX2][CX2,CX3]){1,}[O,N;H1]' for input: '[OX2][CX2,CX3]([OX2][CX2,CX3]){1,}[O,N;H1]'
[21:16:25] SMARTS Parse Error: syntax error while parsing: [CX3](=O)[CX2]{1,}[CX3](=O)
[21:16:25] SMARTS Parse Error: Failed parsing SMARTS '[CX3](=O)[CX2]{1,}[CX3](=O)' for input: '[CX3](=O)[CX2]{1,}[CX3](=O)'
[21:16:25] SMARTS Parse 

LOO 8
LOO 9
LOO 10
LOO 11
LOO 12


[21:16:26] SMARTS Parse Error: syntax error while parsing: [c][c](c[c][c][c])[$(F)_100
[21:16:26] SMARTS Parse Error: Failed parsing SMARTS '[c][c](c[c][c][c])[$(F)_100' for input: '[c][c](c[c][c][c])[$(F)'
[21:16:26] SMARTS Parse Error: syntax error while parsing: $(Cl)_100
[21:16:26] SMARTS Parse Error: Failed parsing SMARTS '$(Cl)_100' for input: '$(Cl)'
[21:16:26] SMARTS Parse Error: syntax error while parsing: $(Br)_100
[21:16:26] SMARTS Parse Error: Failed parsing SMARTS '$(Br)_100' for input: '$(Br)'
[21:16:26] SMARTS Parse Error: syntax error while parsing: $(I)_100
[21:16:26] SMARTS Parse Error: Failed parsing SMARTS '$(I)_100' for input: '$(I)'
[21:16:26] SMARTS Parse Error: syntax error while parsing: [CX4](F)(F)F]
[21:16:26] SMARTS Parse Error: Failed parsing SMARTS '[CX4](F)(F)F]' for input: '[CX4](F)(F)F]'
[21:16:26] SMARTS Parse Error: syntax error while parsing: [c][c](c[c][c][c])[$(O[CH3])_100
[21:16:26] SMARTS Parse Error: Failed parsing SMARTS '[c][c](c[c][c][c])[$(O

LOO 13
LOO 14
LOO 15
LOO 16
LOO 17
LOO 18
LOO 19
LOO 20
LOO 21


...

LOO 22
LOO 23
LOO 24
LOO 25
LOO 26
LOO 27
LOO 28


[21:16:26] SMARTS Parse Error: syntax error while parsing: [cH]-[c](:[cH]):[c]([CX3](=O)[H],C(F)(F)F,C=O)-[cH]:[cH]-[CX3](=O)[OX2H1]
[21:16:26] SMARTS Parse Error: Failed parsing SMARTS '[cH]-[c](:[cH]):[c]([CX3](=O)[H],C(F)(F)F,C=O)-[cH]:[cH]-[CX3](=O)[OX2H1]' for input: '[cH]-[c](:[cH]):[c]([CX3](=O)[H],C(F)(F)F,C=O)-[cH]:[cH]-[CX3](=O)[OX2H1]'
[21:16:26] SMARTS Parse Error: syntax error while parsing: [SX2H][C;!R][C;!R]?[C;!R]?
[21:16:26] SMARTS Parse Error: Failed parsing SMARTS '[SX2H][C;!R][C;!R]?[C;!R]?' for input: '[SX2H][C;!R][C;!R]?[C;!R]?'
[21:16:26] SMARTS Parse Error: syntax error while parsing: ([#6][OX2])+[#6]
[21:16:26] SMARTS Parse Error: Failed parsing SMARTS '([#6][OX2])+[#6]' for input: '([#6][OX2])+[#6]'
[21:16:26] SMARTS Parse Error: syntax error while parsing: (c1ccc(N)cc1)
[21:16:26] SMARTS Parse Error: Failed parsing SMARTS '(c1ccc(N)cc1)' for input: '(c1ccc(N)cc1)'
[21:16:26] SMARTS Parse Error: syntax error while parsing: (c1ccc([N+](=O)[O-])cc1)
[21:16:26] S

LOO 29
LOO 30
LOO 31
LOO 32
LOO 33


[21:16:26] SMARTS Parse Error: syntax error while parsing: [CH3][CH2]{0,6}C(=O)O
[21:16:26] SMARTS Parse Error: Failed parsing SMARTS '[CH3][CH2]{0,6}C(=O)O' for input: '[CH3][CH2]{0,6}C(=O)O'
[21:16:26] SMARTS Parse Error: syntax error while parsing: [CH3][CH2]{0,6}C(=O)O
[21:16:26] SMARTS Parse Error: Failed parsing SMARTS '[CH3][CH2]{0,6}C(=O)O' for input: '[CH3][CH2]{0,6}C(=O)O'
[21:16:26] SMARTS Parse Error: syntax error while parsing: ([#6][OX2][#6]){2,}
[21:16:26] SMARTS Parse Error: Failed parsing SMARTS '([#6][OX2][#6]){2,}' for input: '([#6][OX2][#6]){2,}'


LOO 34
LOO 35
LOO 36
Groups of columns with identical values:
['LOO_1_Rule 1', 'LOO_10_Rule 2', 'LOO_22_Rule 7', 'LOO_29_Rule 3', 'LOO_31_Rule 3', 'LOO_34_Rule 6']
['LOO_1_Rule 2', 'LOO_13_Rule 1']
['LOO_1_Rule 3', 'LOO_1_Rule 5', 'LOO_1_Rule 10', 'LOO_1_Rule 11', 'LOO_1_Rule 13', 'LOO_1_Rule 14', 'LOO_2_Rule 1', 'LOO_2_Rule 3', 'LOO_2_Rule 5', 'LOO_2_Rule 5.1', 'LOO_2_Rule 10', 'LOO_2_Rule 10.1', 'LOO_2_Rule 13', 'LOO_3_Rule 3', 'LOO_3_Rule 4', 'LOO_3_Rule 13', 'LOO_3_Rule 14', 'LOO_4_Rule 3', 'LOO_4_Rule 9', 'LOO_4_Rule 10', 'LOO_5_Rule 3', 'LOO_5_Rule 4', 'LOO_5_Rule 9', 'LOO_5_Rule 11', 'LOO_5_Rule 14', 'LOO_6_Rule 2', 'LOO_6_Rule 3', 'LOO_6_Rule 6', 'LOO_7_Rule 1', 'LOO_7_Rule 7', 'LOO_8_Rule 3', 'LOO_8_Rule 6', 'LOO_8_Rule 10', 'LOO_8_Rule 12', 'LOO_8_Rule 2', 'LOO_8_Rule 9', 'LOO_8_Rule 14', 'LOO_9_Rule 7', 'LOO_9_Rule 8', 'LOO_9_Rule 9', 'LOO_9_Rule 10', 'LOO_10_Rule 3', 'LOO_10_Rule 10', 'LOO_10_Rule 11', 'LOO_10_Rule 12', 'LOO_11_Rule 2', 'LOO_11_Rule 5', 'LOO_11_Rule 10', 'L

## Exp. + Traditional Descriptor

In [31]:
# Load the target column
target = 'yield'
df = pd.read_csv("agent/data/data.csv")[[target]]

# Calculate the median of the target column
median_value = df[target].median()

# Transform the target column into 0/1 based on the median
df[target] = (df[target] > median_value).astype(int)

# Load the dataset
dataset = pd.read_csv("agent/data/data.csv")


# Convert the labels to numpy array
label = df[target].values

feature_file = "agent/data/del_and_exp_features.csv"
selected_features = ['Fe_loading','Solvent_area','Fun_LogP_Max']

feature = pd.read_csv(feature_file)[selected_features]
feature = feature.values


model_cla = RandomForestClassifier(n_estimators=500, max_depth=3, max_features=None, random_state=42, n_jobs=64)
metrics = calculate_metrics_with_loo(feature,label,model_cla)
print('Traditional + Exp. Descriptor')
print(metrics)

Traditional + Exp. Descriptor
{'recall': 0.7777777777777778, 'precision': 0.9333333333333333, 'f1_score': 0.8484848484848485, 'roc_auc_score': 0.8981481481481481}
